# DCGAN on FashionMNIST
This notebook implements a Deep Convolutional GAN to generate synthetic FashionMNIST images.

## Install dependencies

In [ ]:
!pip install torch torchvision tqdm tensorboard

## Configuration

In [ ]:
class Config:
    DATA_DIR    = "data"
    OUTPUT_DIR  = "outputs"
    LOG_DIR     = "logs"
    IMG_SIZE    = 28
    CHANNELS    = 1
    LATENT_DIM  = 100
    BATCH_SIZE  = 64
    LR          = 1e-4
    BETAS       = (0.5, 0.999)
    EPOCHS      = 50
    SAVE_EPOCHS = {10, 30, 50}
    SEED        = 42

## Imports

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image, make_grid
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

## Dataset Loader

In [ ]:
def get_dataloader(cfg):
    tf = transforms.Compose([
        transforms.Resize(cfg.IMG_SIZE),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,)),
    ])
    ds = datasets.FashionMNIST(
        root=cfg.DATA_DIR, train=True, download=True, transform=tf
    )
    return DataLoader(ds, batch_size=cfg.BATCH_SIZE, shuffle=True, num_workers=4)

## Models

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        feat, slope, drop = 64, 0.3, 0.3
        self.conv = nn.Sequential(
            nn.Conv2d(cfg.CHANNELS, feat, 5, 2, 2, bias=False),
            nn.LeakyReLU(slope, inplace=True),
            nn.Dropout(drop),
            nn.Conv2d(feat, feat*2, 5, 2, 2, bias=False),
            nn.LeakyReLU(slope, inplace=True),
            nn.Dropout(drop),
        )
        ds = cfg.IMG_SIZE // 4
        self.fc = nn.Linear(feat*2*ds*ds, 1, bias=False)
    def forward(self, x):
        x = self.conv(x)
        return self.fc(x.view(x.size(0), -1)).view(-1)

class Generator(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        feat, slope = 64, 0.3
        ds = cfg.IMG_SIZE // 4
        self.fc = nn.Sequential(
            nn.Linear(cfg.LATENT_DIM, feat*4*ds*ds, bias=False),
            nn.BatchNorm1d(feat*4*ds*ds),
            nn.LeakyReLU(slope, inplace=True),
        )
        self.deconv = nn.Sequential(
            nn.Unflatten(1, (feat*4, ds, ds)),
            nn.ConvTranspose2d(feat*4, feat*2, 5, 2, 2, output_padding=1, bias=False),
            nn.BatchNorm2d(feat*2),
            nn.LeakyReLU(slope, inplace=True),
            nn.ConvTranspose2d(feat*2, feat, 5, 2, 2, output_padding=1, bias=False),
            nn.BatchNorm2d(feat),
            nn.LeakyReLU(slope, inplace=True),
            nn.ConvTranspose2d(feat, cfg.CHANNELS, 5, 1, 2, bias=False),
            nn.Tanh(),
        )
    def forward(self, z):
        x = self.fc(z)
        return self.deconv(x)

## Initialization & Utils

In [ ]:
def weights_init(m):
    if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.Linear)):
        nn.init.normal_(m.weight, 0.0, 0.02)

def save_samples(gen, epoch, fixed_noise, cfg):
    gen.eval()
    with torch.no_grad():
        imgs = gen(fixed_noise).add(1).div(2)
        grid = make_grid(imgs, nrow=8)
        os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
        save_image(grid, f"{cfg.OUTPUT_DIR}/epoch_{epoch}.png")
    gen.train()

## Training

In [ ]:
def train(cfg):
    torch.manual_seed(cfg.SEED)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    dl = get_dataloader(cfg)
    gen = Generator(cfg).to(device)
    disc = Discriminator(cfg).to(device)
    gen.apply(weights_init); disc.apply(weights_init)
    opt_g = optim.Adam(gen.parameters(), lr=cfg.LR, betas=cfg.BETAS)
    opt_d = optim.Adam(disc.parameters(), lr=cfg.LR, betas=cfg.BETAS)
    criterion = nn.BCEWithLogitsLoss()
    fixed_noise = torch.randn(64, cfg.LATENT_DIM, device=device)
    writer = SummaryWriter(cfg.LOG_DIR)
    step = 0
    for ep in range(1, cfg.EPOCHS+1):
        for real, _ in tqdm(dl, desc=f"Epoch {ep}/{cfg.EPOCHS}"):
            real = real.to(device); bs = real.size(0)
            noise = torch.randn(bs, cfg.LATENT_DIM, device=device)
            fake = gen(noise)
            # Discriminator
            opt_d.zero_grad()
            d_real = disc(real)
            d_fake = disc(fake.detach())
            loss_d = criterion(d_real, torch.ones_like(d_real)) + criterion(d_fake, torch.zeros_like(d_fake))
            loss_d.backward(); opt_d.step()
            # Generator
            opt_g.zero_grad()
            d_fake2 = disc(fake)
            loss_g = criterion(d_fake2, torch.ones_like(d_fake2))
            loss_g.backward(); opt_g.step()
            writer.add_scalar("Loss/Discriminator", loss_d.item(), step)
            writer.add_scalar("Loss/Generator",     loss_g.item(), step)
            step += 1
        if ep in cfg.SAVE_EPOCHS:
            save_samples(gen, ep, fixed_noise, cfg)
    torch.save(gen.state_dict(), f"{cfg.OUTPUT_DIR}/generator.pth")
    torch.save(disc.state_dict(), f"{cfg.OUTPUT_DIR}/discriminator.pth")
    writer.close()

cfg = Config()
train(cfg)

## Generate Samples

In [ ]:
def generate(cfg, model_path=None, n=64):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    G = Generator(cfg).to(device)
    mp = model_path or f"{cfg.OUTPUT_DIR}/generator.pth"
    G.load_state_dict(torch.load(mp, map_location=device))
    noise = torch.randn(n, cfg.LATENT_DIM, device=device)
    save_samples(G, "final", noise, cfg)

cfg = Config()
generate(cfg)